In [168]:
import pandas as pd
import numpy as np
from translate import translator
import time
import random
from babel import Locale
import json
import operator
from prompter import yesno

In [126]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /home/sara/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
path = "Data/"
path_foreigners = "Resident foreigners on 1st January - Citizenship/resident_foreigners.csv"
path_resident = "Resident population  on 1st January /resident.csv"

In [3]:
resident_foreigners = pd.read_table(path+path_foreigners)
resident_foreigners = resident_foreigners.drop_duplicates()
resident_foreigners = resident_foreigners.drop("Tipo di indicatore demografico", axis = 1)
resident_foreigners.columns = ["Territorio", "Cittadinanza", "Periodo", "Sesso", "Numero"]
resident_foreigners.head()

,Territorio,Cittadinanza,Periodo,Sesso,Numero
0,Italia,Andorra,2013,maschi,0
1,Italia,Andorra,2014,maschi,2
2,Italia,Andorra,2015,maschi,1
3,Italia,Andorra,2016,maschi,1
4,Italia,Andorra,2017,maschi,1


In [4]:
# Is not useful to keep the info about the ages, since it's an info not available for the foreigners table
resident = pd.read_table(path+path_resident)
resident = resident.drop_duplicates()
resident = resident[resident["Età"] == "totale"]
resident = resident.drop(["Tipo di indicatore demografico", "Stato civile", "Età"], axis = 1)
resident.columns = ["Territorio", "Periodo", "Sesso", "Numero"]
resident.head()

,Territorio,Periodo,Sesso,Numero
179734,Piemonte,2013,maschi,2110576
179735,Piemonte,2014,maschi,2146570
179736,Piemonte,2015,maschi,2140294
179737,Piemonte,2016,maschi,2131892
179738,Piemonte,2017,maschi,2129403


In [5]:
provincia_regione = pd.read_table(path+"regioni.csv", sep = "\t")
provincia_regione.head()

,Provincia,Regione
0,Torino,Piemonte
1,Vercelli,Piemonte
2,Novara,Piemonte
3,Cuneo,Piemonte
4,Asti,Piemonte


In [6]:
regione_zona = pd.read_table(path+"territori.csv", sep = "\t")
regione_zona.head()

,Regione,Zona
0,Abruzzo,Sud
1,Basilicata,Sud
2,Calabria,Sud
3,Campania,Sud
4,Emilia-Romagna,Nord-est


Check if the aggragation is right.. that is: 
- the sum over the municipalities (comuni) conrespond to the aggragation value at the region (regione) level;
- the sum over the regions conrespond to the aggregation value at the zone (nord, sud, etc..) level;
- for each tuple (Territorio, Cittadinanza, Anno) if maschio + femmina = totale.

Check both the resident and the resident_foreigners

In [7]:
regioni = list(set(regione_zona["Regione"]))
zone = list(set(regione_zona["Zona"]))
territori = list(set(resident_foreigners["Territorio"]))
years = list(set(resident_foreigners["Periodo"]))
origine = list(set(resident_foreigners["Cittadinanza"]))

### Resident Foreigners Check

In [8]:
temp = resident_foreigners.copy()
# keep only the municipalities and the region data
territori_1 = list(set(list(provincia_regione["Provincia"]) + list(provincia_regione["Regione"])))
temp = temp.loc[[i in territori_1 for i in temp["Territorio"]]]
# do not consider the Valle d'Aosta / Vallée d'Aoste data since the aggregate and the singular data coincide
temp = temp[temp["Territorio"] != "Valle d'Aosta / Vallée d'Aoste"]
# change the sign to the "total" elements
temp["Numero"][[i in list(set(provincia_regione["Regione"])) for i in temp["Territorio"]]] = -temp["Numero"][[i in list(set(provincia_regione["Regione"])) for i in temp["Territorio"]]]

regioni = list(set(provincia_regione["Regione"]))
province = list(set(provincia_regione["Provincia"]))
# add the regione field, so we can aggregate on it
temp["Regione"] = [provincia_regione["Regione"][provincia_regione["Provincia"] == i].values[0] if i in province else i for i in temp["Territorio"]]

# sum all over the same (Regione, Cittadinanza, Periodo, Sesso)
temp['totale'] = temp.groupby(["Regione", "Cittadinanza", "Periodo", "Sesso"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Region-Aggregation works on 'resident_foreigners'!")

Region-Aggregation works on 'resident_foreigners'!


In [9]:
'''
for r in regioni:
    print(r)
    for o in origine:   
        comuni = comune_regione[comune_regione["Regione"] == r]["Comune"]
        for y in years:
            temp = 0
            for c in comuni:
                try:
                    temp += resident_foreigners[(resident_foreigners["Territorio"] == c) & (resident_foreigners["Cittadinanza"] == o) & (resident_foreigners["Sesso"] == "totale") & (resident_foreigners["Periodo"] == y)]["Numero"].values
                # Handle missing data
                except ValueError:
                    pass
            if temp != resident_foreigners[(resident_foreigners["Territorio"] == r) & (resident_foreigners["Cittadinanza"] == o) & (resident_foreigners["Sesso"] == "totale") & (resident_foreigners["Periodo"] == y)]["Numero"].values:
                print(temp)
'''

'\nfor r in regioni:\n    print(r)\n    for o in origine:   \n        comuni = comune_regione[comune_regione["Regione"] == r]["Comune"]\n        for y in years:\n            temp = 0\n            for c in comuni:\n                try:\n                    temp += resident_foreigners[(resident_foreigners["Territorio"] == c) & (resident_foreigners["Cittadinanza"] == o) & (resident_foreigners["Sesso"] == "totale") & (resident_foreigners["Periodo"] == y)]["Numero"].values\n                # Handle missing data\n                except ValueError:\n                    pass\n            if temp != resident_foreigners[(resident_foreigners["Territorio"] == r) & (resident_foreigners["Cittadinanza"] == o) & (resident_foreigners["Sesso"] == "totale") & (resident_foreigners["Periodo"] == y)]["Numero"].values:\n                print(temp)\n'

In [10]:
temp = resident_foreigners.copy()
# keep only the municipalities and the region data
territori_1 = list(set(list(regione_zona["Regione"]) + list(regione_zona["Zona"])))
temp = temp.loc[[i in territori_1 for i in temp["Territorio"]]]

# change the sign to the "total" elements
temp["Numero"][[i in list(set(regione_zona["Zona"])) for i in temp["Territorio"]]] = -temp["Numero"][[i in list(set(regione_zona["Zona"])) for i in temp["Territorio"]]]

regioni = list(set(provincia_regione["Regione"]))
zone = list(set(regione_zona["Zona"]))

# add the regione field, so we can aggregate on it
temp["Zona"] = [regione_zona["Zona"][regione_zona["Regione"] == i].values[0] if i in regioni else i for i in temp["Territorio"]]

# sum all over the same (Regione, Cittadinanza, Periodo, Sesso)
temp['totale'] = temp.groupby(["Zona", "Cittadinanza", "Periodo", "Sesso"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Zone-Aggregation works on 'resident_foreigners'!")

Zone-Aggregation works on 'resident_foreigners'!


In [11]:
'''
for z in zone:
    print(z)
    for o in origine:   
        regioni = regione_zona[regione_zona["Zona"] == z]["Regione"]
        for y in years:
            temp = 0
            for r in regioni:
                try:
                    temp += resident_foreigners[(resident_foreigners["Territorio"] == r) & (resident_foreigners["Cittadinanza"] == o) & (resident_foreigners["Sesso"] == "totale") & (resident_foreigners["Periodo"] == y)]["Numero"].values
                # Handle missing data
                except ValueError:
                    pass
            if temp != resident_foreigners[(resident_foreigners["Territorio"] == z) & (resident_foreigners["Cittadinanza"] == o) & (resident_foreigners["Sesso"] == "totale") & (resident_foreigners["Periodo"] == y)]["Numero"].values:
                print(temp)
'''

'\nfor z in zone:\n    print(z)\n    for o in origine:   \n        regioni = regione_zona[regione_zona["Zona"] == z]["Regione"]\n        for y in years:\n            temp = 0\n            for r in regioni:\n                try:\n                    temp += resident_foreigners[(resident_foreigners["Territorio"] == r) & (resident_foreigners["Cittadinanza"] == o) & (resident_foreigners["Sesso"] == "totale") & (resident_foreigners["Periodo"] == y)]["Numero"].values\n                # Handle missing data\n                except ValueError:\n                    pass\n            if temp != resident_foreigners[(resident_foreigners["Territorio"] == z) & (resident_foreigners["Cittadinanza"] == o) & (resident_foreigners["Sesso"] == "totale") & (resident_foreigners["Periodo"] == y)]["Numero"].values:\n                print(temp)\n'

In [11]:
temp = resident_foreigners.copy()
# change the sign to the "total" elements
temp["Numero"][temp[temp["Sesso"] == "totale"].index] = -temp["Numero"][temp[temp["Sesso"] == "totale"].index]
# sum all over the same (Territorio, Cittadinanza, Periodo)
temp['totale'] = temp.groupby(["Territorio", "Cittadinanza", "Periodo"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Aggregation maschio + femmina works on 'resident_foreigners'!")

Aggregation maschio + femmina works on 'resident_foreigners'!


/home/sara/myenv/lib64/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### Resident Check

In [12]:
temp = resident.copy()
# keep only the municipalities and the region data
territori_1 = list(set(list(provincia_regione["Provincia"]) + list(provincia_regione["Regione"])))
temp = temp.loc[[i in territori_1 for i in temp["Territorio"]]]
# do not consider the Valle d'Aosta / Vallée d'Aoste data since the aggregate and the singular data coincide
temp = temp[temp["Territorio"] != "Valle d'Aosta / Vallée d'Aoste"]
# change the sign to the "total" elements
temp["Numero"][[i in list(set(provincia_regione["Regione"])) for i in temp["Territorio"]]] = -temp["Numero"][[i in list(set(provincia_regione["Regione"])) for i in temp["Territorio"]]]

regioni = list(set(provincia_regione["Regione"]))
province = list(set(provincia_regione["Provincia"]))
# add the regione field, so we can aggregate on it
temp["Regione"] = [provincia_regione["Regione"][provincia_regione["Provincia"] == i].values[0] if i in province else i for i in temp["Territorio"]]

# sum all over the same (Regione, Periodo, Sesso)
temp['totale'] = temp.groupby(["Regione", "Periodo", "Sesso"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Region-Aggregation works on 'resident'!")

Region-Aggregation works on 'resident'!


In [13]:
temp = resident.copy()
# keep only the region and the zone data
territori_1 = list(set(list(regione_zona["Regione"]) + list(regione_zona["Zona"])))
temp = temp.loc[[i in territori_1 for i in temp["Territorio"]]]

# change the sign to the "total" elements
temp["Numero"][[i in list(set(regione_zona["Zona"])) for i in temp["Territorio"]]] = -temp["Numero"][[i in list(set(regione_zona["Zona"])) for i in temp["Territorio"]]]

regioni = list(set(provincia_regione["Regione"]))
zone = list(set(regione_zona["Zona"]))

# add the regione field, so we can aggregate on it
temp["Zona"] = [regione_zona["Zona"][regione_zona["Regione"] == i].values[0] if i in regioni else i for i in temp["Territorio"]]

# sum all over the same (Regione, Periodo, Sesso)
temp['totale'] = temp.groupby(["Zona", "Periodo", "Sesso"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Zone-Aggregation works on 'resident'!")

Zone-Aggregation works on 'resident'!


In [14]:
temp = resident.copy()
# change the sign to the "total" elements
temp["Numero"][temp[temp["Sesso"] == "totale"].index] = -temp["Numero"][temp[temp["Sesso"] == "totale"].index]
# sum all over the same (Territorio, Periodo)
temp['totale'] = temp.groupby(["Territorio", "Periodo"])['Numero'].transform(np.sum)

# the sum should always be 0
if len(list(set(temp["totale"]))) == 1 and list(set(temp["totale"]))[0] == 0:
    print("Aggregation maschio + femmina works on 'resident'!")

Aggregation maschio + femmina works on 'resident'!


/home/sara/myenv/lib64/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


The aggragation checks are verified. Thus, it's possible to drop duplication to get the normal form.

### .. to Normal Form

In [15]:
resident_foreigners_norm = resident_foreigners.copy() 

# drop region and zone data
territori_1 = set(list(regione_zona["Regione"]) + (list(regione_zona["Zona"]) + list(["Italia"]))) 
resident_foreigners_norm = resident_foreigners_norm.loc[[i not in territori_1 for i in resident_foreigners_norm["Territorio"]]]
# drop total (mashio+femmina) data
resident_foreigners_norm = resident_foreigners_norm[resident_foreigners_norm["Sesso"] != "totale"]

resident_foreigners_norm.index = range(len(resident_foreigners_norm))

In [16]:
resident_norm = resident.copy() 

# drop region and zone data
territori_1 = set(list(regione_zona["Regione"]) + (list(regione_zona["Zona"]) + list(["Italia"]))) 
resident_norm = resident_norm.loc[[i not in territori_1 for i in resident_norm["Territorio"]]]
# drop total (mashio+femmina) data
resident_norm = resident_norm[resident_norm["Sesso"] != "totale"]

resident_norm.index = range(len(resident_norm))

In [17]:
# Provincia is now better than generic Territorio
resident_norm.rename(columns = {"Territorio": "Provincia"}, inplace=True)
resident_foreigners_norm.rename(columns = {"Territorio": "Provincia"}, inplace=True)

In [18]:
resident_norm.head()

,Provincia,Periodo,Sesso,Numero
0,Torino,2013,maschi,1084528
1,Torino,2014,maschi,1109048
2,Torino,2015,maschi,1105608
3,Torino,2016,maschi,1101427
4,Torino,2017,maschi,1100562


In [19]:
resident_foreigners_norm.head()

,Provincia,Cittadinanza,Periodo,Sesso,Numero
0,Alessandria,Andorra,2017,maschi,0
1,Alessandria,Andorra,2017,femmine,1
2,Genova,Andorra,2013,maschi,0
3,Genova,Andorra,2013,femmine,1
4,Como,Andorra,2013,maschi,0


In [20]:
# dump
resident_norm.to_csv("Data_norm/resident_norm.csv")
resident_foreigners_norm.to_csv("Data_norm/resident_foreigners_norm.csv")

### Distances

In [21]:
# country specific table
geo_info = pd.read_table(path+"geo_cepii.tsv", sep = "\t")
geo_info = geo_info[['iso2', 'iso3', 'country', 'area', 'dis_int', 'landlocked', 'continent', 'langoff_1', 
                     'langoff_2', 'langoff_3', 'lang20_1', 'lang20_2', 'lang20_3', 'lang20_4', 'lang9_1', 'lang9_2', 
                     'lang9_3', 'lang9_4', 'colonizer1', 'colonizer2', 'colonizer3','colonizer4', 'short_colonizer1', 
                     'short_colonizer2','short_colonizer3']]
# pair of countries distances table
geo_dist = pd.read_table(path+"dist_cepii.tsv", sep = "\t", na_values = {"distw": '.', "distwces": '.'})
geo_dist = geo_dist[['iso_o', 'iso_d', 'contig', 'dist', 'distcap', 'distw', 'distwces']]

In [22]:
geo_info["dis_int"] = [float(geo_info["dis_int"][i].replace(",", ".")) for i in geo_info.index]
geo_info["area"] = geo_info["area"].astype(int)

geo_dist["dist"] = [float(geo_dist["dist"][i].replace(",", ".")) for i in geo_dist.index]
geo_dist["distcap"] = [float(geo_dist["distcap"][i].replace(",", ".")) for i in geo_dist.index]
geo_dist["distw"] = [float(geo_dist["distw"][i].replace(",", ".")) if geo_dist["distw"][i] is not np.nan else np.nan for i in geo_dist.index]
geo_dist["distwces"] = [float(geo_dist["distwces"][i].replace(",", ".")) if geo_dist["distwces"][i] is not np.nan else np.nan for i in geo_dist.index]

In [23]:
geo_info.head()

,iso2,iso3,country,area,dis_int,landlocked,continent,langoff_1,langoff_2,langoff_3,...,lang9_2,lang9_3,lang9_4,colonizer1,colonizer2,colonizer3,colonizer4,short_colonizer1,short_colonizer2,short_colonizer3
0,AW,ABW,Aruba,193,5.225315,0,America,Dutch,Spanish,.,...,.,.,.,NLD,.,.,.,.,.,.
1,AF,AFG,Afghanistan,652225,303.761400,1,Asia,Persian,.,.,...,Uzbek,.,.,.,.,.,.,GBR,.,.
2,AO,AGO,Angola,1246700,419.966600,0,Africa,Portuguese,.,.,...,.,.,.,PRT,.,.,.,.,.,.
3,AI,AIA,Anguilla,102,3.798690,0,America,English,.,.,...,.,.,.,GBR,.,.,.,.,.,.
4,AL,ALB,Albania,28748,63.773110,0,Europe,Albanian,.,.,...,.,.,.,TUR,.,.,.,.,.,.


In [24]:
geo_dist.head()

,iso_o,iso_d,contig,dist,distcap,distw,distwces
0,ABW,ABW,0,5.225315,5.225315,25.09354,23.04723
1,ABW,AFG,0,13257.810000,13257.810000,13168.22000,13166.37000
2,ABW,AGO,0,9516.913000,9516.913000,9587.31600,9584.19300
3,ABW,AIA,0,983.268200,983.268200,976.89740,976.89160
4,ABW,ALB,0,9091.742000,9091.742000,9091.57600,9091.46600


Geo-distances variables:
- iso_o, iso_d: ISO codes in three characters;
- contig: dummy, whether the two countries are contiguous;
- comlang_off: dummy, whether the two countries share the same official language;
- comcol: dummy, whether the two countries have had a common colonizer after 1945;
- colony: dummy, whether the two countries have had a colonial link;
- col45: dummy, whether the two countries have had a colonial relationship after tionship 1945;
- curcol: dummy, whether the two countries are currently in a colonial relationship;
- smctry: dummy, whether the two countries were/are the same country;
- dist: simple distance between the most important cities (in terms of population) of the two countries;
- distcap: simple distance between the most capital cities of the two countries;
- distw, distwces: weighted distances; distance between the largest cities of the two countries - those inter cities distances are weighted by the share of the city in the overall country's population.


Problem: there is no matching between the countries in the resident_foreigner and the those in the geo_info. In the first table info are in italian, in the second in english. A tranlation is needed.

In [33]:
# iso2 of all countries
iso2 = {geo_info["country"][i]: geo_info["iso2"][i] for i in geo_info.index}

# translate the countries using babel
countries_translation = {}
not_found = []
locale = Locale('it')
for c in iso2.keys():
    try:
        countries_translation[locale.territories[iso2[c]]] = c
        #countries_translation[c] = locale.territories[iso2[c]]
    except:
        not_found.append(c)

# translate the countries not found by babel (just 5) using google translate
for c in not_found:
    time.sleep(random.randint(1,10))
    countries_translation[translator("en", "it", c)[0][0][0]] = c

HTTPError: 503 Server Error: Service Unavailable for url: https://translate.google.com/translate_a/single?client=a&ie=utf-8&oe=utf-8&dt=t&sl=en&tl=it&q=Netherland+Antilles

In [43]:
with open('Data/countries_translation.json', 'w') as j:
    json.dump(countries_translation, j)

In [35]:
countries_translation = json.load(open('Data/countries_translation.json'))

In [46]:
#countries_translation = {y:x for x,y in countries_translation.items()}
#countries_translation

In [58]:
resident_foreigners_norm["Country"] = [countries_translation[c] if c in countries_translation.keys() else '-' for c in resident_foreigners_norm["Cittadinanza"]]

In [88]:
resident_foreigners_norm = resident_foreigners_norm.drop('Country', axis = 1)

In [117]:
temp = []
countries_translation_update = {}
for c in resident_foreigners_norm["Cittadinanza"]:
    try:
        temp.append(countries_translation[c])
    except:
        try:
            # in some case, the country name used in italian is the english one
            temp.append(list(countries_translation.keys())[list(countries_translation.values()).index(c)])
        except:
            temp_split = c.replace('(', ';').replace(')', ';').split(';')
            try:
                temp.append(countries_translation[temp_split[0]])
            except:
                try:
                    temp.append(countries_translation[temp_split[1]])
                except:
                    temp.append('-')

        
resident_foreigners_norm["Country"] = temp

In [205]:
# all the country whose matching ita-eng is ok!
countries_translation_update = {}
not_assigned_countries = []
for c in list(set(resident_foreigners_norm["Cittadinnanza"])):
    try:
        countries_translation_update[c] = countries_translation[c]
    except:
        not_assigned_countries.append(c)

In [206]:
(' --> ').join((c, max(temp.items(), key=operator.itemgetter(1))[0]))

'Mali --> Territori palestinesi'

In [207]:
# for all not assigned
not_yet_assigned_countries = []
update = []
for c in not_assigned_countries[:10]:
    not_assigned_keys = list((set(countries_translation.keys()) - set(countries_translation_update.keys())) - set(update))
    # find the most similar not yet assigne keys:
    temp = {n: similar(c, n) for n in not_assigned_keys}
    #print(c, max(temp.items(), key=operator.itemgetter(1))[0])
    question = (' --> ').join((c, max(temp.items(), key=operator.itemgetter(1))[0]))
    if yesno(question) == True:
        update = [max(temp.items(), key=operator.itemgetter(1))[0]]
        countries_translation_update[c] = update
    else:
        not_yet_assigned_countries.append(c)

Corea del sud --> Corea del Sud [Y/n] y
Liechtenstein --> Benin [Y/n] n
Bahrain --> Bahrein [Y/n] y
Myanmar (Ex Birmania) --> Myanmar (Birmania) [Y/n] y
Costa d'Avorio --> Costa d’Avorio [Y/n] y
Taiwan (ex Formosa) --> Taiwan [Y/n] y
Regno unito --> Regno Unito [Y/n] y
Kosovo --> Costa d’Avorio [Y/n] n
Macedonia, Ex Repubblica Jugoslava di --> Congo (Repubblica Democratica del) [Y/n] n
Territori dell'Autonomia Palestinese --> Territori palestinesi [Y/n] y


In [210]:
countries_translation_update.keys()

dict_keys(['Bahamas', 'Kirghizistan', 'Nuova Zelanda', 'Papua Nuova Guinea', 'Antigua e Barbuda', 'Mauritius', 'Azerbaigian', 'Danimarca', 'Polonia', 'Gambia', 'Paesi Bassi', 'Antille Olandesi', 'Canada', 'Finlandia', 'Camerun', 'Georgia', 'Andorra', 'Slovenia', 'Suriname', 'Thailandia', 'Yemen', 'Australia', 'Ecuador', 'Burundi', 'Perù', 'Giappone', 'Nepal', 'Zambia', 'Slovacchia', 'Palau', 'Namibia', 'Ucraina', 'Ghana', 'Francia', 'Kenya', 'Sudan', 'Norvegia', 'Cipro', 'Cambogia', 'Cina', 'Messico', 'Senegal', 'Ungheria', 'Emirati Arabi Uniti', 'Portogallo', 'Guatemala', 'Arabia Saudita', 'Indonesia', 'Comore', 'Honduras', 'Figi', 'Mongolia', 'Ruanda', 'Tonga', 'Guyana', 'Samoa', 'Turkmenistan', 'Germania', 'Giamaica', 'Nicaragua', 'Afghanistan', 'Swaziland', 'Cile', 'Bolivia', 'Togo', 'Ciad', 'Brasile', 'Bielorussia', 'Malta', 'Bhutan', 'Malawi', 'Siria', 'Barbados', 'Filippine', 'Turchia', 'Guinea', 'Libano', 'Saint Kitts e Nevis', 'Uganda', 'Niger', 'Svizzera', 'Israele', 'Singapo

In [211]:
update

['Territori palestinesi']

In [215]:
set(countries_translation.keys()) - set(update)

{'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua e Barbuda',
 'Antille Olandesi',
 'Arabia Saudita',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaigian',
 'Bahamas',
 'Bahrein',
 'Bangladesh',
 'Barbados',
 'Belgio',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bielorussia',
 'Bolivia',
 'Bosnia ed Erzegovina',
 'Botswana',
 'Brasile',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambogia',
 'Camerun',
 'Canada',
 'Capo Verde',
 'Ciad',
 'Cile',
 'Cina',
 'Cipro',
 'Colombia',
 'Comore',
 'Congo (Repubblica Democratica del)',
 'Congo-Brazzaville',
 'Corea del Nord',
 'Corea del Sud',
 'Costa Rica',
 'Costa d’Avorio',
 'Croazia',
 'Cuba',
 'Danimarca',
 'Dominica',
 'Ecuador',
 'Egitto',
 'El Salvador',
 'Emirati Arabi Uniti',
 'Eritrea',
 'Estonia',
 'Etiopia',
 'Figi',
 'Filippine',
 'Finlandia',
 'Francia',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germania',
 'Ghana',
 'Giamaica',
 'Giappone',
 'Gibilterra',
 'Gibuti'

In [186]:
not_yet_assigned_countries

['Liechtenstein',
 'Kosovo',
 'Macedonia, Ex Repubblica Jugoslava di',
 "Iran, Repubblica islamica dell'",
 'Corea, Repubblica Popolare Democratica (Corea del Nord)',
 'Riconosciuti non-cittadini (Lettonia)',
 'Mondo',
 'Santa Sede (Stato della Città del Vaticano)',
 'Dominicana, Repubblica',
 'Sud Sudan, Repubblica del',
 'Monaco']

In [187]:
countries_translation_update

{'Afghanistan': 'Afghanistan',
 'Albania': 'Albania',
 'Algeria': 'Algeria',
 'Andorra': 'Andorra',
 'Angola': 'Angola',
 'Antigua e Barbuda': 'Antigua and Barbuda',
 'Antille Olandesi': 'Netherland Antilles',
 'Arabia Saudita': 'Saudi Arabia',
 'Argentina': 'Argentina',
 'Armenia': 'Armenia',
 'Australia': 'Australia',
 'Austria': 'Austria',
 'Azerbaigian': 'Azerbaijan',
 'Bahamas': 'Bahamas',
 'Bahrain': 'Bahrein',
 'Bangladesh': 'Bangladesh',
 'Barbados': 'Barbados',
 'Belgio': 'Belgium and Luxembourg',
 'Belize': 'Belize',
 'Benin (ex Dahomey)': 'Benin',
 'Bhutan': 'Bhutan',
 'Bielorussia': 'Belarus',
 'Bolivia': 'Bolivia',
 'Bosnia-Erzegovina': 'Bosnia ed Erzegovina',
 'Botswana': 'Botswana',
 'Brasile': 'Brazil',
 'Brunei Darussalam': 'Brunei',
 'Bulgaria': 'Bulgaria',
 'Burkina Faso (ex Alto Volta)': 'Burkina Faso',
 'Burundi': 'Burundi',
 'Cambogia': 'Cambodia',
 'Camerun': 'Cameroon',
 'Canada': 'Canada',
 'Capo Verde': 'Cape Verde',
 'Ceca, Repubblica': 'Repubblica Ceca',
 'C

In [66]:
list(countries_translation.keys())[list(countries_translation.values()).index('Saudi Arabia')]

'Arabia Saudita'

In [120]:
countries_translation

{'Afghanistan': 'Afghanistan',
 'Albania': 'Albania',
 'Algeria': 'Algeria',
 'Andorra': 'Andorra',
 'Angola': 'Angola',
 'Anguilla': 'Anguilla',
 'Antigua e Barbuda': 'Antigua and Barbuda',
 'Antille Olandesi': 'Netherland Antilles',
 'Arabia Saudita': 'Saudi Arabia',
 'Argentina': 'Argentina',
 'Armenia': 'Armenia',
 'Aruba': 'Aruba',
 'Australia': 'Australia',
 'Austria': 'Austria',
 'Azerbaigian': 'Azerbaijan',
 'Bahamas': 'Bahamas',
 'Bahrein': 'Bahrain',
 'Bangladesh': 'Bangladesh',
 'Barbados': 'Barbados',
 'Belgio': 'Belgium and Luxembourg',
 'Belize': 'Belize',
 'Benin': 'Benin',
 'Bermuda': 'Bermuda',
 'Bhutan': 'Bhutan',
 'Bielorussia': 'Belarus',
 'Bolivia': 'Bolivia',
 'Bosnia ed Erzegovina': 'Bosnia and Herzegovina',
 'Botswana': 'Botswana',
 'Brasile': 'Brazil',
 'Brunei': 'Brunei Darussalam',
 'Bulgaria': 'Bulgaria',
 'Burkina Faso': 'Burkina Faso',
 'Burundi': 'Burundi',
 'Cambogia': 'Cambodia',
 'Camerun': 'Cameroon',
 'Canada': 'Canada',
 'Capo Verde': 'Cape Verde',


In [114]:
countries_translation[list(set(resident_foreigners_norm[resident_foreigners_norm["Country"] == '-']["Cittadinanza"]))[4].replace('(', ';').replace(')', ';').split(';')[1]]

"Korea, Dem. People's Rep. of"

In [132]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

similar('Corea del sud', 'Corea del Sud')

0.9230769230769231

In [119]:
list(set(resident_foreigners_norm[resident_foreigners_norm["Country"] == '-']["Cittadinanza"]))

['Bosnia-Erzegovina',
 'Corea del sud',
 'Macedonia, Ex Repubblica Jugoslava di',
 'Moldova',
 'Liechtenstein',
 "Territori dell'Autonomia Palestinese",
 'Zimbabwe (ex Rhodesia)',
 'Myanmar (Ex Birmania)',
 'Centrafricana, Repubblica',
 'Burkina Faso (ex Alto Volta)',
 'Congo, Repubblica democratica del (ex Zaire)',
 'Saint Vincent e Grenadine',
 'São Tomé e Principe',
 'Guinea equatoriale',
 'Benin (ex Dahomey)',
 'Ceca, Repubblica',
 "Costa d'Avorio",
 'Taiwan (ex Formosa)',
 'Santa Sede (Stato della Città del Vaticano)',
 'Santa Lucia',
 'Dominicana, Repubblica',
 'Kazakhstan',
 'Sri Lanka (ex Ceylon)',
 'Regno unito',
 'Kosovo',
 'Sud Sudan, Repubblica del',
 'Sud Africa',
 'Monaco',
 "Iran, Repubblica islamica dell'",
 'Congo (Repubblica del)',
 'Serbia, Repubblica di',
 'Mondo',
 'Montenegro',
 'Marshall, Isole',
 'Timor-Leste',
 'Salomone, Isole']

In [23]:
# consider only the distances involving Italy
italia_iso = "ITA"
geo_dist_ita = geo_dist[geo_dist["iso_o"] == "ITA"]
geo_dist_ita.index = range(len(geo_dist_ita))

# store the once egual both in italian and english - in order to have less requests to google translate
countries = list(set(resident_foreigners_norm["Cittadinanza"]))
#countries_en = {i: i for i in countries if i in geo_info["country"].values} 
# define the list of countries to translate
#countries_to_translate = list(countries - countries_en.keys())

In [47]:
contries_en_translation = countries_en.copy()

In [ ]:
countries_iso = [geo_info[geo_info["country"] == i]["iso3"].values[0] for i in countries]

### Population info

In [183]:
pop = pd.read_table(path+"API_SP.POP.TOTL_DS2_en_csv_v2/population.csv", sep = ",", skiprows = 4)
# 2017 data all nan (not yet available)
pop = pop[['Country Name', 'Country Code', '2010', '2011', '2012', '2013','2014','2015', '2016']]
# rename the columns - coherence between the tables
pop.rename(columns = {'Country Name': 'country', 'Country Code': 'iso3'}, inplace = True)

pop.head()

,country,iso3,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0
1,Afghanistan,AFG,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0
2,Angola,AGO,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0
3,Albania,ALB,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0
4,Andorra,AND,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0
